# \[TS\] CH03. 평활법 실습

김보람  
2023-10-06

> 해당 자료는 전북대학교 이영미 교수님 2023고급시계열분석 자료임

# 패키지

In [ ]:
############## package
library(forecast) #ses
library(data.table)
library(ggplot2)
library(lmtest) #dwtest
library(TTR) #SMA


In [ ]:
options(repr.plot.width = 15, repr.plot.height = 8)


# 이동평균법

## Kings data

In [ ]:
kings=scan("http://robjhyndman.com/tsdldata/misc/kings.dat",skip=3)

In [ ]:
kingstimeseries = ts(kings)
plot(kingstimeseries, lwd=2)


In [ ]:
## window=3
kingstimeseriesSMA3 <- SMA(kingstimeseries,n=3)
plot.ts(kingstimeseries, lwd=2)
lines(kingstimeseriesSMA3, col='red', lty=2, lwd=2)
legend("topleft",
 legend=c("original", expression(m==3)), In [68]: In [2]: In [3]: In [4]: In [5]: In [6]:
 col=c("black","red"),
 lty=c(1,2), lwd=2)

In [ ]:
kingstimeseries

In [ ]:
kingstimeseriesSMA3

In [ ]:
## window=3 vs. 7
plot.ts(kingstimeseries, lwd=2)
lines(kingstimeseriesSMA3, col='red', lty=2, lwd=2)
lines( SMA(kingstimeseries,n=10), col='blue', lty=2, lwd=2)
legend("topleft",
 legend=c("original", expression(m==3), expression(m==10)),
 col=c("black","red","blue"),
 lty=c(1,2,2), lwd=2)

### 모형 평가

In [ ]:
## m=3
mean((kings- kingstimeseriesSMA3)^2, na.rm=T) ##MSE
mean(abs(kings- kingstimeseriesSMA3), na.rm=T) ##MAE
mean(abs(kings- kingstimeseriesSMA3/kings), na.rm=T)*100 ##MAPE

In [ ]:
## m=10
mean((kings- SMA(kingstimeseries,n=10))^2, na.rm=T) ##MSE
mean(abs(kings- SMA(kingstimeseries,n=10)), na.rm=T) ##MAE
mean(abs(kings- SMA(kingstimeseries,n=10)/kings), na.rm=T)*100 ##MAPE

## mindex data

In [ ]:
z <- scan("mindex.txt")
mindex <- ts(z, start = c(1986, 1), frequency = 12)
mindex

In [ ]:
mindex_sma3 <- SMA(mindex,n=3)
mindex_sma10 <- SMA(mindex,n=10)

In [ ]:
plot.ts(mindex, lwd=2)
lines(mindex_sma3, col='red', lty=2, lwd=2)
lines(mindex_sma10, col='blue', lty=2, lwd=2)
legend("topright",
     legend=c("original", expression(m==3), expression(m==10)),
     col=c("black","red","blue"),
     lty=c(1,2,2), lwd=2)

# 단순지수평활법

In [ ]:
z <- scan("mindex.txt")
mindex <- ts(z, start = c(1986, 1), frequency = 12)


In [ ]:
tmp.dat <- data.frame(day = seq.Date(as.Date("1986-01-01"),
 by='month',
 length.out=length(z)), 
              ind = z)
head(tmp.dat)
                     

In [ ]:
ggplot(tmp.dat, aes(day, ind))+geom_line(col='skyblue', lwd=2) +
 geom_point(col='steelblue', cex=2)+
 ggtitle("중간재 출하지수")+
 theme_bw()+
 theme(plot.title = element_text(size=30),
 axis.title = element_blank())


단수지수평활 $\alpha=0.3$

In [ ]:
fit0 <- HoltWinters(mindex,
 alpha=0.3,
 beta=FALSE,
 gamma=FALSE)


In [ ]:
fit0

In [ ]:
ls(fit0)


In [ ]:
head(fit0$fitted)

In [ ]:
fit0$SSE

In [ ]:
plot(fit0, lwd=3, cex.main = 2)
legend("topright", legend=c("observed", "ses"), lty=1,lwd=c(1,3), col=1:2)

In [ ]:
fit01 <- ses(mindex,
 alpha = 0.3,
 initial = 'simple',
 h = 10) ##Exponential smoothing forecasts


In [ ]:
fit01

In [ ]:
ls(fit01)


In [ ]:
summary(fit01)

In [ ]:
plot(fit01,, cex.main = 2)

## 여러 평활상수에 따른 변화

In [ ]:
w <-c(seq(0.1,0.8,0.1), seq(0.81, 0.99, 0.01))

In [ ]:
SSE_ses <- sapply(w, function(alpha) HoltWinters(mindex,
                 alpha=alpha,
                 beta=FALSE,
                 gamma=FALSE)$SSE)
plot(w,SSE_ses, type="o", xlab="weight", ylab="SSE", pch=16,
     main="1 시차 후 예측오차의 제곱합",
     cex.main = 2, cex.lab=2)


In [ ]:
which.min(SSE_ses)

In [ ]:
w[which.min(SSE_ses)]

In [ ]:
plot(w[-(1:7)],SSE_ses[-(1:7)], type="o", xlab="weight", ylab="SSE", pch=16,
 main="1 시차 후 예측오차의 제곱합",
 cex.main = 2, cex.lab=2)


In [ ]:
fit1 <- ses(mindex, alpha=w[which.min(SSE_ses)], initial = "simple", h=6)

In [ ]:
plot(fit1, xlab="", ylab="",
 main="중간재 출하지수와 단순지수평활값 alpha=0.9",
 lty=1,col="black",
 cex.main = 2, cex.lab=2 )
lines(fitted(fit1), col="red", lty=2)
legend("topright", legend=c("Mindex", expression(alpha==0.9)),
 lty=1:2,col=c("black","red"))


In [ ]:
fit0_w <- HoltWinters(mindex,
 beta=FALSE,
 gamma=FALSE)
fit0_w


## 잔차

In [ ]:
plot(fit1$residuals, ylab="residual",
 main="예측 오차의 시계열그림 : alpha=0.9",
 cex.main = 2, cex.lab=2); abline(h=0)


오차의 평균이 0인가?

In [ ]:
t.test(fit1$residual)


오차가 독립인가?

In [ ]:
dwtest(lm(fit1$residual~1))

In [ ]:
fit2 <- ses(mindex, alpha=0.3, h=6)
plot(fit2, xlab="year", ylab="mindex",
 main=expression("중간재 출하지수와 단순지수평활값 "~alpha==0.3),
 lty=1,col="black",
 cex.main = 2,
 cex.lab=2)
lines(fitted(fit2), col="red", lty=2)
legend("topright",
 legend=c("Mindex",expression(alpha==0.3)),
 lty=1:2,
 col=c("black","red"))


In [ ]:
plot(fit2$residuals, ylab="residual",
 main=expression("중간재 출하지수와 단순지수평활값 "~alpha==0.3),
 cex.main = 2,
 cex.lab=2); abline(h=0)


오차의 평균이 0인가?

In [ ]:
t.test(fit2$residual)

오차가 독립인가?

In [ ]:
dwtest(lm(fit2$residual~1))

# 이중지수평활법

In [ ]:
z <- scan("stock.txt")
stock <- ts(z, start=c(1984,1), frequency=12)


In [ ]:
plot(stock, main = '월별주가지수', cex.main=2, cex.lab=2)

## 1모수 이중지수평활 $(\alpha = \beta)$

In [ ]:
fit4 = holt(stock, alpha=0.6, beta=0.6, h=6)
fit4$model

In [ ]:
# plot(fit4, ylab="", xlab="", lty=1, col="black",
#  main="주가지수와 이중지수평활값 : alpha=beta=0.6"
#  )
# lines(fitted(fit4), col="red", lty=2)
# legend("topleft", lty=1:2, col=c("black","red"), c("Index", "Double"), bty = "n"

## 이중지수평활 $\alpha, \beta$ 추정

In [ ]:
fit5 = holt(stock, h=6)
fit5$model


In [ ]:
# plot(fit5, ylab="Index", xlab="year", lty=1, col="black",
#  main=expression("중간재 출하지수와 이중지수평활값 : "~alpha~","~beta~ " : es
#  cex.main=2)
# lines(fitted(fit5), col="red", lty=2)
# legend("topleft", lty=1:2, col=c("black","red"), c("Index", "Double"))


In [ ]:
# plot(resid(fit5), main=expression("Residual Plot : "~alpha~","~beta~ " : estimat
#  cex.main=2)
# abline(h=0)

# 계절지수평활

In [ ]:
z <- scan("koreapass.txt")
pass <- ts(z, start=c(1981,1), frequency=12)


In [ ]:
plot.ts(pass)


## Holt Winters additive model

In [ ]:
fit_hw <- HoltWinters(pass)

In [ ]:
fit_hw

In [ ]:
plot(fit_hw, lwd=2)


In [ ]:
predict(fit_hw, n.ahead=12, prediction.interval = T, level=0.95)


In [ ]:
fit6= hw(pass,
 alpha = fit_hw$alpha,
 beta = fit_hw$beta,
 gamma = fit_hw$gamma,
 seasonal="additive",
 initial="simple",
 h=12)
fit6$model

In [ ]:
fit6


In [ ]:
plot(fit6, ylab="passenger", xlab="year", lty=1, col="blue",
 main="Winters 가법계절지수평활된 자료의 시계열 그림",
 cex.main=2)


In [ ]:
ts.plot(fit6$residual, ylab="residual",
 main="가법모형의 잔차 그림",
 cex.main=2); abline(h=0)


In [ ]:
dwtest(lm(fit6$residual~1), alternative = 'two.sided')

## Holt Winters multiplicative model

In [ ]:
fit_hw_m <- HoltWinters(pass, seasonal="multiplicative")
fit_hw_m


In [ ]:
predict(fit_hw_m, n.ahead=12, prediction.interval = T, level=0.95)


In [ ]:
fit7= hw(pass,
 alpha = fit_hw_m$alpha,
 beta = fit_hw_m$beta,
 gamma = fit_hw_m$gamma,
 seasonal="additive",
 initial="simple",
 h=12)
fit7$model


In [ ]:
plot(fit7, ylab="passenger", xlab="year", lty=1, col="blue",
 main="Winters 승법계절지수평활된 자료의 시계열 그림",
 cex.main=2)
lines(fit7$fitted, col="red", lty=2)
legend("topleft", lty=1:2, col=c("blue","red"), c("Pass", "Multiplicative"))

In [ ]:
ts.plot(fit7$residual, ylab="residual",
 main="승법모형의 잔차 그림",
 cex.main=2); abline(h=0)


In [ ]:
dwtest(lm(fit7$residual~1), alternative = 'two.sided')